
## Deeplearning


In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from PIL import Image
import matplotlib.pylab as plt

#plt.rcParams['figure.figsize'] = (10, 12.0)

NUM_HADAMARD = 256

sort_index = pd.read_csv('F:/Code/signal-signal_fromscratch/data/collect_sorted_index.csv')
sort_index = np.array(sort_index)
sort_index = sort_index[:,1:]
sort_index = [x-1 for x in sort_index]

RD_dir = u'C:/Users/zh/Desktop/hadamard/32RD.png'
RD=Image.open(RD_dir)
RD = np.array(RD)
CC = np.ones((1024,1024))
RD_p = 2*RD - CC
RD_anti = -1 * RD_p

def load_data(data_dir):
    data = np.array(pd.read_csv(data_dir))
    return data[:,1:]

def save_data(x,data_dir):
    save_data = pd.DataFrame(x)
    save_data.to_csv(data_dir)
    
    
def save_model(sess,ckpt_dir,global_step,model_name='signaltosignal'):
    saver = tf.train.Saver()
    checkpoint_dir = ckpt_dir
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    print("Saving model....")
    saver.save(sess,os.path.join(checkpoint_dir,model_name),global_step = global_step)

def load_model(ckpt_dir):
    print("Loading model...")
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        full_path = tf.train.latest_checkpoint(ckpt_dir)
        global_step = int(full_path.split('/')[-1].split('-')[-1])
        saver.restore(sess,full_path)
        return True,global_step
    else :
        return False,0


    
def save_data(x,data_dir):
    save_data = pd.DataFrame(x)
    save_data.to_csv(data_dir)    

def save_image(file_name,original_image,generate_image,noise_image):
    original_image = normalize(np.squeeze(original_image))
    generate_image = normalize(np.squeeze(generate_image))
    noise_image = normalize(np.squeeze(noise_image))
    
    image = np.concatenate([original_image,generate_image,noise_image],axis = 1)
    im = Image.fromarray(image.astype('uint8')).convert('L')
    im.save(file_name, 'png')

def normalize(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))*255   
    
def inference(input_data):
    with tf.variable_scope('fc1') as scope:
    #    X = tf.placeholder(tf.float32,[1 256],name = 'X1')
        weights = tf.get_variable('w1',
                              shape = [256,1024],
                              dtype = tf.float32,
                              initializer = tf.truncated_normal_initializer(stddev=0.03,dtype=tf.float32))
        bias = tf.get_variable('b1',shape=[1024],dtype = tf.float32 ,initializer = tf.constant_initializer(0.1))
        Y = tf.add(tf.matmul(input_data,weights),bias)
        output1 = tf.nn.relu(Y,name = scope.name)
    
    with tf.variable_scope('fc2') as scope:
        weights = tf.get_variable(name = 'w2',
                                  shape = [1024,1024],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.03,dtype = tf.float32))
        bias = tf.get_variable(name ='b2',
                              shape = [1024],
                              dtype = tf.float32,
                              initializer = tf.constant_initializer(0.1))
        output2 = tf.nn.relu(tf.add(tf.matmul(output1,weights),bias),name = scope.name)
        
    with tf.variable_scope('fc3') as scope:
        weights = tf.get_variable(name = 'w3',
                                  shape = [1024,256],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.03,dtype = tf.float32))
        bias = tf.get_variable(name ='b3',
                              shape = [256],
                              dtype = tf.float32,
                              initializer = tf.constant_initializer(0.1))
        tf.summary.histogram('bias',bias)
        output2 = tf.add(tf.matmul(output1,weights),bias,name = scope.name)
  
    return output2


    
    
    
    
def train(Xdata , Ydata ,Xevaldata,Yevaldata,sort_index,RD_p, learning_rate = 0.0001, epoch = 20 , batch_size = 32):
    
#    tf.reset_default_graph()
    logs = r"F:/Code/signal-signal_fromscratch/logs/"
    ckpt = r"F:/Code/signal-signal_fromscratch/checkpoints/"
    sample_dir = r'F:/Code/signal-signal_fromscratch/data/evaluate'
    X = tf.placeholder(dtype = tf.float32, shape =[None,256] ,name = 'X')
    Y = tf.placeholder(name = 'Y', shape =[None,256] ,dtype = tf.float32)
    
    output = inference(X)
    
    losses = tf.reduce_mean(tf.square(output-Y))
    
    tf.summary.scalar('loss',losses)


    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(losses) 
    
    init = tf.global_variables_initializer()
    print("Start training ...")
    
    print("Please wait for a while ...")
    with tf.Session() as sess:
        
        sess.run(init)
        iter_num = 0
        merged = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(logs,sess.graph)
        
        numBatch = np.int(Xdata.shape[0]/batch_size)
        for i in range(epoch):
            for batch_id in range(numBatch):
                batch_Xdata = Xdata[batch_id*batch_size:(batch_id+1)*batch_size,:]
                batch_Ydata = Ydata[batch_id*batch_size:(batch_id+1)*batch_size,:]
                _ , loss,summary = sess.run([train_op , losses,merged] , feed_dict ={X:batch_Xdata ,Y:batch_Ydata })
                iter_num += 1
                summary_writer.add_summary(summary,iter_num)
            if i%2 == 0:
                print("epoch %d loss %.4f " % (i,loss))
            if i%10 == 0:
                print("Evaluating ...")
      
                for j in range(Xevaldata.shape[0]):
                    data = [Xevaldata[j][:]]
                    eval_out = sess.run(output,feed_dict = {X:data})
                    
                    sum1 = np.zeros((32,32))
                    sum2 = np.zeros((32,32))
                    sum3 = np.zeros((32,32))
                    y = 0
                    for k in np.squeeze(eval_out):
                        y = y+1
                        pattern = RD_p[sort_index[y-1]].reshape((1024,1)) 
                        sum1 = sum1 + Xevaldata[j][y-1] * pattern.reshape((32,32))
                        sum2 = sum2 + Yevaldata[j][y-1] * pattern.reshape((32,32))
                        sum3 = sum3 + k * pattern.reshape((32,32))
                    sum1[0,0] = np.mean(sum1)  
                    sum2[0,0] = np.mean(sum2)  
                    sum3[0,0] = np.mean(sum3)  
          
                    save_image(os.path.join(sample_dir, 'test%d_%d.png' % (y, iter_num)),sum2,sum1,sum3)
            
                save_model(sess,ckpt,iter_num)
        
 

In [ ]:
np.random.seed(1)
train_data_X = np.random.randn(40000,256) * 100
train_data_Y = train_data_X + np.random.randn(40000,256)*2

BATCH_SIZE = 64 

train(train_data_X ,train_data_Y)

print("The train has been done . -_-")


## test

In [ ]:
import tensorflow as tf
import numpy as np

#weights = tf.get_variable('w1',[2,1],initializer=tf.truncated_normal_initializer(stddev=0.03,dtype=tf.float32) )
xx = np.random.randn(1,256).astype(np.float32)
Y = inference(xx)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)
p = sess.run(Y)
p

In [ ]:
x = tf.Variable([1,3],'w')

In [ ]:
a= 1
b= 1
assert a==b,' a!=b '

## generate data

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt

#plt.rcParams['figure.figsize'] = (10, 12.0)

NUM_HADAMARD = 256

def normalize(y):
    a,b=np.shape(y)
    n = np.zeros((a,b))
    for i in range(a):
        for j in range(b):
            n[i,j] = np.int(y[i,j])
    return n


RD_dir = u'C:/Users/zh/Desktop/hadamard/32RD.png'
RD=Image.open(RD_dir)
RD = np.array(RD)
CC = np.ones((1024,1024))
RD_p = 2*RD - CC
RD_anti = -1 * RD_p
#RD_n = -1 * RD_p
# K = 256
# RD_use =np.vstack((RD_p[0:K,:],RD_n[0:K,:]))  #创造需要使用的 Hadamard 矩阵
#NUM_PICTURE = 42000
#RATIO = 0.1
DIM = 1024 
i = np.random.randint(100)


#rand_pattern = np.random.randn(DIM,DIM)
signals = []
for i in range(200):
    filename1 = 'F:\Code\kaggle\Digit Recognizer\image\image' + '%d' % i+ '.jpg'
    x = Image.open(filename1)
    image_test = x.resize((32,32))
    image_test = np.array(image_test)
    pic = image_test.reshape((1,1024))
    sum1 = np.zeros((32,32))
    signal = []
    for j in range(1024):
        RD_pattern = RD_p[j].reshape((1024,1))
        anti_pattern = RD_anti[j].reshape((1024,1))
        pattern = RD_p[j].reshape((1024,1)) 
        temp = np.dot(pic,RD_pattern) - np.dot(pic,anti_pattern)
        signal.append(temp)
        sum1 = sum1 + temp * pattern.reshape((32,32))
    signals.append(signal)
#sum1 = normalize(((sum1 - np.min(sum1))/(np.max(sum1) - np.min(sum1)))*256)
#image_ghost = Image.fromarray(sum1)
#image_ghost = image_ghost.convert('L')
sum1[0,0] = np.mean(sum1)
plt.figure(figsize=(8,10))

plt.subplot(211)
plt.title("Sequence Signal",fontsize =18)
plt.xlabel("Hadamard Row")
plt.ylabel("Idensity")
plt.plot(np.squeeze(signal))

plt.subplot(212)
signal_sorted = sorted(np.abs(signal),reverse=True)
plt.title("Sorted Signal",fontsize =18)
plt.plot(np.squeeze(signal_sorted))

plt.tight_layout()
plt.show()

print("****************** ")
print("****************** ")
print("****************** ")

print("Get the bigger signal ... ")

signal_abs = np.abs(np.squeeze(signal))
signal_dict = {}
j= 0
for i in signal_abs:
    j = j+1
    signal_dict[j] = i 

xxx =sorted(signal_dict.items(),key = lambda items:items[1],reverse = True)
signal_picture = [i[0] for i in xxx]


sum2 = np.zeros((32,32))
sum3 = np.zeros((32,32))
signal = np.squeeze(signal)
for j in signal_picture[:NUM_HADAMARD]:
    RD_pattern = RD_p[j-1].reshape((1024,1))
    anti_pattern = RD_anti[j-1].reshape((1024,1))
    pattern = RD_p[j-1].reshape((1024,1)) 
    temp = np.dot(pic,RD_pattern) - np.dot(pic,anti_pattern)
    sum2 = sum2 + temp * pattern.reshape((32,32))
y = 0
for k in signal[:NUM_HADAMARD]:
    y= y+1
    pattern = RD_p[y-1].reshape((1024,1)) 
    sum3 = sum3 + k * pattern.reshape((32,32))
#sum1 = normalize(((sum1 - np.min(sum1))/(np.max(sum1) - np.min(sum1)))*256)
#image_ghost = Image.fromarray(sum1)
#image_ghost = image_ghost.convert('L')
sum2[0,0] = np.mean(sum2)
sum3[0,0] = np.mean(sum3)
plt.figure()

plt.subplot(131)
plt.title("Full Samples",fontsize =18)
plt.imshow(sum1,cmap = plt.cm.gray)
plt.axis('off')

plt.subplot(132)
plt.title("Bigger "+str(NUM_HADAMARD),fontsize =18)

plt.imshow(sum2,cmap = plt.cm.gray)
plt.axis('off')
plt.subplot(133)
plt.title("First " +str(NUM_HADAMARD),fontsize =18)
plt.imshow(sum3,cmap = plt.cm.gray)
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
import pandas as pd 

#plt.rcParams['figure.figsize'] = (10, 12.0)

NUM_HADAMARD = 64

def normalize(y):
    a,b=np.shape(y)
    n = np.zeros((a,b))
    for i in range(a):
        for j in range(b):
            n[i,j] = np.int(y[i,j])
    return n


RD_dir = u'C:/Users/zh/Desktop/hadamard/32RD.png'
RD=Image.open(RD_dir)
RD = np.array(RD)
CC = np.ones((1024,1024))
RD_p = 2*RD - CC
RD_anti = -1 * RD_p
#RD_n = -1 * RD_p
# K = 256
# RD_use =np.vstack((RD_p[0:K,:],RD_n[0:K,:]))  #创造需要使用的 Hadamard 矩阵
#NUM_PICTURE = 42000
#RATIO = 0.1
DIM = 1024 
#i = np.random.randint(100)


#rand_pattern = np.random.randn(DIM,DIM)
signals = []
for i in range(40000):
    filename1 = 'F:\Code\kaggle\Digit Recognizer\image\image' + '%d' % i+ '.jpg'
    x = Image.open(filename1)
    image_test = x.resize((32,32))
    image_test = np.array(image_test)
    pic = image_test.reshape((1,1024))
    sum1 = np.zeros((32,32))
    signal = []
    for j in range(1024):
        RD_pattern = RD_p[j].reshape((1024,1))
        anti_pattern = RD_anti[j].reshape((1024,1))
        pattern = RD_p[j].reshape((1024,1)) 
        temp = np.dot(pic,RD_pattern) - np.dot(pic,anti_pattern)
        signal.append(temp)
        sum1 = sum1 + temp * pattern.reshape((32,32))
    if i%2000 == 0:
        print("Have finished %d signal" %i)
    signal = np.squeeze(signal)
    signals.append(signal)

#sig = np.squeeze(signals)
data_full_samples = pd.DataFrame(data = signals)
data_full_samples.to_csv('F:/Code/signal-signal_fromscratch/data/data_full_samples.csv')

print("Signals have been written in CSV")

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
import pandas as pd 
x = pd.read_csv('F:/Code/signal-signal_fromscratch/data/data_full_samples.csv')
original_signals =np.array(x)
original_signals =original_signals[:,1:]
print("We get the original data")

signals_full_sample = np.abs(original_signals)


def normalize(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))*255

signals_normalized = []
for i in range(signals_full_sample.shape[0]):
    one_of_signals = normalize(signals_full_sample[i])
    signals_normalized.append(one_of_signals)
    

signals_normalized = np.array(signals_normalized)
print(signals_normalized.shape)

print("Signals normalize have been finished ")
preSort_signals = np.sum(signals_normalized,axis=0)

#signal_abs = np.abs(np.squeeze(signal))
signal_dict = {}
j= 0
for i in preSort_signals:
    j = j+1
    signal_dict[j] = i 

xxx =sorted(signal_dict.items(),key = lambda items:items[1],reverse = True)
collect_sorted_signal= [i[0] for i in xxx]
print("We get the collective sorted picture index !")

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt

#plt.rcParams['figure.figsize'] = (10, 12.0)

NUM_HADAMARD = 256

def normalize(y):
    a,b=np.shape(y)
    n = np.zeros((a,b))
    for i in range(a):
        for j in range(b):
            n[i,j] = np.int(y[i,j])
    return n


RD_dir = u'C:/Users/zh/Desktop/hadamard/32RD.png'
RD=Image.open(RD_dir)
RD = np.array(RD)
CC = np.ones((1024,1024))
RD_p = 2*RD - CC
RD_anti = -1 * RD_p
sum2 = np.zeros((32,32))
for j in collet_sorted_signal[:NUM_HADAMARD]:
    RD_pattern = RD_p[j-1].reshape((1024,1))
    anti_pattern = RD_anti[j-1].reshape((1024,1))
    pattern = RD_p[j-1].reshape((1024,1)) 
    temp = np.dot(pic,RD_pattern) - np.dot(pic,anti_pattern)
    sum2 = sum2 + temp * pattern.reshape((32,32))
    
plt.title("Bigger "+str(NUM_HADAMARD),fontsize =18)

plt.imshow(sum2,cmap = plt.cm.gray)
plt.axis('off')
plt.show()

In [ ]:
haha = pd.DataFrame(collect_sorted_signal)
haha.to_csv('F:/Code/signal-signal_fromscratch/data/collect_sorted_index.csv')

In [ ]:
xx = [1,3,2,0,6,4,5]
xxx =np.array([2,3,4,5,6,7,8])
xxx[xx]

In [ ]:
collect_sorted_signal1 = [x-1 for x in collect_sorted_signal]
sorted_signals =original_signals[:,collect_sorted_signal1]
sorted_signals_pd = pd.DataFrame(sorted_signals)
sorted_signals_pd.to_csv('F:/Code/signal-signal_fromscratch/data/sorted_signals.csv')

In [ ]:
testx =original_signals[3,collect_sorted_signal1]
plt.plot(np.abs(testx))
plt.show()

In [ ]:
import numpy as np
import matplotlib.pylab as plt

def GWN(x,snr):
    snr = 10**(snr/10)
    xpower = np.sum(x**2)/len(x)
    npower = xpower/snr
    return x + np.random.randn(len(x))*np.sqrt(npower)

#  SNR = 10*log10(xpower/npower) 
#
def cal_snr(x,y,datatype = 'signal'):
    if datatype == 'signal':
        return 10*np.log10(np.mean(x**2)/np.mean((x-y)**2))
    if datatype == 'image':
        return 10*np.log10(255**2/np.mean((x-y)**2))
   
voice = np.linspace(1,5,1000)
y = (2)**voice
y_noise = GWN(y,10)
plt.plot(voice,y,color = 'b')
plt.plot(voice,y_noise,color = 'y', linestyle =':')
plt.show()

print(cal_snr(y,y_noise))

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
import pandas as pd 
x = pd.read_csv('F:/Code/signal-signal_fromscratch/data/sorted_signals.csv')
original_signals =np.array(x)
original_signals =original_signals[:,1:]
print("We get the original data")
def normalize(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))*255

noise_signals = []
for i in range(original_signals.shape[0]):
    one_of_signals = normalize(original_signals[i,:])
    one_of_signals_noise = GWN(one_of_signals,20)
    noise_signals.append(one_of_signals_noise)
    


In [ ]:
print(cal_snr(normalize(original_signals[6,:]),noise_signals[6][:]))   

In [ ]:
xixi = pd.DataFrame(noise_signals)
xixi.to_csv('F:/Code/signal-signal_fromscratch/data/sorted_noise_signals.csv')

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt

#plt.rcParams['figure.figsize'] = (10, 12.0)

NUM_HADAMARD = 1024

def normalize(y):
    a,b=np.shape(y)
    n = np.zeros((a,b))
    for i in range(a):
        for j in range(b):
            n[i,j] = np.int(y[i,j])
    return n
sort_index = pd.read_csv('F:/Code/signal-signal_fromscratch/data/collect_sorted_index.csv')
sort_index = np.array(sort_index)
sort_index = sort_index[:,1:]
sort_index = [x-1 for x in sort_index]
RD_dir = u'C:/Users/zh/Desktop/hadamard/32RD.png'
RD=Image.open(RD_dir)
RD = np.array(RD)
CC = np.ones((1024,1024))
RD_p = 2*RD - CC
RD_anti = -1 * RD_p
sum3 = np.zeros((32,32))
y = 0
noise_signals_1 = noise_signals[1][:]
for k in noise_signals_1[:NUM_HADAMARD]:
    y= y+1
    pattern = RD_p[sort_index[y-1]].reshape((1024,1)) 
    sum3 = sum3 + k * pattern.reshape((32,32))
sum3[0,0] = np.mean(sum3)  
plt.title("First " +str(NUM_HADAMARD),fontsize =18)
plt.imshow(sum3,cmap = plt.cm.gray)
plt.axis('off')
plt.show()

In [ ]:

sort_index

In [2]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
import pandas as pd 


x = load_data('F:/Code/signal-signal_fromscratch/data/sorted_noise_signals.csv')
y = load_data('F:/Code/signal-signal_fromscratch/data/sorted_signals.csv')

x_train_data = x[:39895,:256]
y_train_data = y[:39895,:256]
x_test_data = x[39895:39995,:256]
y_test_data = y[39895:39995,:256]
x_eval_data = x[39995:,:256]
y_eval_data = y[39995:,:256]
BATCH_SIZE = 64 

train(x_train_data,y_train_data,x_eval_data,y_eval_data,sort_index,RD_p,epoch = 50)

print("The train has been done . -_-")

Start training ...
Please wait for a while ...
epoch 0 loss 54932928.0000 
Evaluating ...
Saving model....
epoch 2 loss 51637160.0000 
epoch 4 loss 47069548.0000 
epoch 6 loss 43355916.0000 
epoch 8 loss 40146968.0000 
epoch 10 loss 37158736.0000 
Evaluating ...
Saving model....
epoch 12 loss 34604984.0000 
epoch 14 loss 32583748.0000 
epoch 16 loss 30936788.0000 
epoch 18 loss 29510492.0000 
epoch 20 loss 28245358.0000 
Evaluating ...
Saving model....
epoch 22 loss 27114424.0000 
epoch 24 loss 26105124.0000 
epoch 26 loss 25201544.0000 
epoch 28 loss 24386972.0000 
epoch 30 loss 23660536.0000 
Evaluating ...
Saving model....
epoch 32 loss 23002950.0000 
epoch 34 loss 22398088.0000 
epoch 36 loss 21846692.0000 
epoch 38 loss 21335368.0000 
epoch 40 loss 20858246.0000 
Evaluating ...
Saving model....
epoch 42 loss 20408012.0000 
epoch 44 loss 19988854.0000 
epoch 46 loss 19591632.0000 
epoch 48 loss 19219078.0000 
The train has been done . -_-


In [ ]:
z = np.random.randn(3,2)
print(z)
np.array([z[1,:]]).shape
np.array(np.squeeze([z[1,:]])).shape
